In [36]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
from PIL import Image, UnidentifiedImageError
import joblib

In [2]:
# Dataset path (Kaggle cats vs dogs train folder)
DATASET_DIR = r"C:\Programming\Prodigy Infotech\Data\PetImages"
BATCH_SIZE = 32
IMG_SIZE = 224
LIMIT = 50000   # limit number of images for demo (increase if you have GPU power)

In [3]:
root_dir = DATASET_DIR

for subdir, _, files in os.walk(root_dir):
    for file in files:
        path = os.path.join(subdir, file)
        try:
            img = Image.open(path)
            img.verify()  # check if corrupted
        except (UnidentifiedImageError, OSError):
            print(f"Deleting corrupted file: {path}")
            os.remove(path)


c:\Facial Recognition\venv\lib\site-packages\PIL\TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [4]:
# Image transformations (resize + normalize for ResNet)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [5]:
# Load dataset (train folder has cat.0.jpg, dog.0.jpg etc.)
full_dataset = datasets.ImageFolder(
    root=DATASET_DIR.rsplit("/", 1)[0],  # parent folder
    transform=transform
)

In [6]:
# Subsample dataset for speed
indices = list(range(len(full_dataset)))
np.random.shuffle(indices)
indices = indices[:LIMIT*2]
subset = torch.utils.data.Subset(full_dataset, indices)

In [7]:
dataloader = DataLoader(subset, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
# Load pretrained ResNet50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
print(device)

cuda


In [10]:
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Identity()  # remove last layer → 2048-d features
resnet = resnet.to(device)
resnet.eval()

c:\Facial Recognition\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Facial Recognition\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
# Extract features
features, labels = [], []
print("Extracting features with ResNet50...")
with torch.no_grad():
    for imgs, lbls in tqdm(dataloader):
        imgs = imgs.to(device)
        outputs = resnet(imgs)       # shape: (batch, 2048)
        features.append(outputs.cpu().numpy())
        labels.append(lbls.numpy())

Extracting features with ResNet50...


100%|██████████| 782/782 [02:13<00:00,  5.86it/s]


In [13]:
efficientnet = models.efficientnet_b0(pretrained=True)
efficientnet.classifier = nn.Identity()   # remove classifier → get 1280-d features
efficientnet = efficientnet.to(device)
efficientnet.eval()

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\ANIMESH/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:24<00:00, 875kB/s] 


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [14]:
features2, labels2 = [], []
print("Extracting features with EfficientNetB0...")
with torch.no_grad():
    for imgs, lbls in tqdm(dataloader):
        imgs = imgs.to(device)
        outputs = efficientnet(imgs)   # shape: (batch, 1280)
        features2.append(outputs.cpu().numpy())
        labels2.append(lbls.numpy())

Extracting features with EfficientNetB0...


100%|██████████| 782/782 [01:29<00:00,  8.77it/s]


In [15]:
X = np.vstack(features)
y = np.hstack(labels)

print("Feature matrix:", X.shape, "Labels:", y.shape)

Feature matrix: (24999, 2048) Labels: (24999,)


In [16]:
X2 = np.vstack(features2)
y2 = np.hstack(labels2)

print("Feature matrix:", X2.shape, "Labels:", y2.shape)

Feature matrix: (24999, 1280) Labels: (24999,)


In [17]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [18]:
# Train-test split
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42, stratify=y2
)

In [19]:
print("Training SVM...")
svm = SVC(kernel="linear", C=1.0)
svm.fit(X_train, y_train)

Training SVM...


,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [20]:
print("Training SVM2 with EfficientNet...")
svm2 = SVC(kernel="linear", C=1.0)
svm2.fit(X2_train, y2_train)

Training SVM2 with EfficientNet...


,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [21]:
print("Evaluating (ResNet50)...")
y_pred = svm.predict(X_test)

Evaluating (ResNet50)...


In [22]:
print("Evaluating with efficientnet...")
y2_pred = svm2.predict(X2_test)

Evaluating with efficientnet...


In [23]:
print("Results for ResNet50 :- ")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))

Results for ResNet50 :- 
Accuracy: 0.9906
              precision    recall  f1-score   support

         Cat       0.99      0.99      0.99      2500
         Dog       0.99      0.99      0.99      2500

    accuracy                           0.99      5000
   macro avg       0.99      0.99      0.99      5000
weighted avg       0.99      0.99      0.99      5000



In [24]:
print("Results for Efficientnet :- ")
print("Accuracy:", accuracy_score(y2_test, y2_pred))
print(classification_report(y2_test, y2_pred, target_names=["Cat", "Dog"]))

Results for Efficientnet :- 
Accuracy: 0.9906
              precision    recall  f1-score   support

         Cat       0.99      0.99      0.99      2500
         Dog       0.99      0.99      0.99      2500

    accuracy                           0.99      5000
   macro avg       0.99      0.99      0.99      5000
weighted avg       0.99      0.99      0.99      5000



In [25]:
print("\n🔹 Training RBF SVM (ResNet50) (slower, better sometimes)...")
svm_rbf = SVC(kernel="rbf", C=10, gamma="scale")
svm_rbf.fit(X_train, y_train)
y_pred_rbf = svm_rbf.predict(X_test)


🔹 Training RBF SVM (ResNet50) (slower, better sometimes)...


In [29]:
print("\n🔹 Training RBF SVM (Efficientnet) (slower, better sometimes)...")
svm2_rbf = SVC(kernel="rbf", C=10, gamma="scale")
svm2_rbf.fit(X2_train, y2_train)



🔹 Training RBF SVM (Efficientnet) (slower, better sometimes)...


,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [30]:
y2_pred_rbf = svm2_rbf.predict(X2_test)
y2_pred_rbf

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [31]:
print("Results for ResNet50 :- ")
print("RBF SVM Accuracy:", accuracy_score(y_test, y_pred_rbf))
print(classification_report(y_test, y_pred_rbf, target_names=["Cat", "Dog"]))

Results for ResNet50 :- 
RBF SVM Accuracy: 0.994
              precision    recall  f1-score   support

         Cat       1.00      0.99      0.99      2500
         Dog       0.99      1.00      0.99      2500

    accuracy                           0.99      5000
   macro avg       0.99      0.99      0.99      5000
weighted avg       0.99      0.99      0.99      5000



In [32]:
print("Results for Efficientnet :- ")
print("RBF SVM Accuracy:", accuracy_score(y_test, y_pred_rbf))
print(classification_report(y_test, y_pred_rbf, target_names=["Cat", "Dog"]))

Results for Efficientnet :- 
RBF SVM Accuracy: 0.994
              precision    recall  f1-score   support

         Cat       1.00      0.99      0.99      2500
         Dog       0.99      1.00      0.99      2500

    accuracy                           0.99      5000
   macro avg       0.99      0.99      0.99      5000
weighted avg       0.99      0.99      0.99      5000



In [37]:
joblib.dump(svm2_rbf, "../Models/svm_resnet.pkl")
print(f"Model Saved to {'Models/svm_resnet.pkl'}")

Model Saved to Models/svm_resnet.pkl
